<a href="https://colab.research.google.com/github/madaan/pie-perf/blob/main/%5BPIE%5D_KNN_Prompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("neulab/codebert-cpp")  # just as an example


No sentence-transformers model found with name /usr0/home/amadaan/.cache/torch/sentence_transformers/neulab_codebert-cpp. Creating a new one with MEAN pooling.


Some weights of RobertaModel were not initialized from the model checkpoint at /usr0/home/amadaan/.cache/torch/sentence_transformers/neulab_codebert-cpp and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import json
from tqdm import tqdm
import numpy as np

In [ ]:
def load_jsonl_data(jsonl_file):
    data = []
    with open(jsonl_file, "r") as f:
        for line in f:
            data.append(json.loads(line))
    return data


In [ ]:
len(training_data)

77967

In [ ]:
import torch

num_gpus = torch.cuda.device_count()
print(f"You have {num_gpus} GPUs available.")


You have 10 GPUs available.


In [ ]:
device = torch.device("cuda:4")


In [ ]:
model.to(device)


SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [ ]:
def encode_in_batches(texts, model, batch_size=256):
    """
    Encode texts in batches using the given model.
    """
    embeddings = []
    for start_idx in tqdm(range(0, len(texts), batch_size)):
        batch_texts = texts[start_idx:start_idx+batch_size]

        # Compute embeddings for the current batch and extend the list of embeddings
        embeddings.extend(model.encode(batch_texts, show_progress_bar=False))

    return np.array(embeddings)


In [ ]:
training_data_src = list(set([x["src_code"] for x in training_data]))
print(len(training_data_src))

21320


In [ ]:
len(training_data)

77967

In [ ]:
src_embeddings = encode_in_batches(training_data_src, model, batch_size=128)


100%|██████████| 167/167 [04:50<00:00,  1.74s/it]


### Build index

In [ ]:
import faiss

dimension = src_embeddings.shape[1]  # Get the size of the embedding
faiss_index = faiss.IndexFlatL2(dimension)
faiss_index.add(np.array(src_embeddings).astype('float32'))


(21320, 768)

### Create prompt

In [ ]:
test_set = load_jsonl_data("/usr1/amadaan/learning2perf/data/gem5/cleaned_test_results.jsonl")
test_queries = [x["src_code"] for x in test_set]


### `num_top_matches` is how we decide the k

In [ ]:
num_top_matches = 4  # or however many top matches you want
top_matches = []
for query_program in tqdm(test_queries):
    query_embedding = model.encode(query_program).astype('float32')
    distances, indices = faiss_index.search(np.array([query_embedding]), num_top_matches)
    top_matches.append(indices)
    # indices will give you the index of the most similar program in your original dataset


100%|██████████| 982/982 [00:41<00:00, 23.90it/s]


In [ ]:
def make_prompt(src_code, tgt_code):
    """
    Make a prompt for a given pair of codes.
    """
    prompt = f"""# slower version:

{src_code}


# optimized version of the same code:

{tgt_code}"""

    return prompt

import re
def canonicalize_newlines(code: str) -> str:
    """
    Canonicalize multiple newlines to a single newline
    """

    # call clang-format to canonicalize newlines

    return re.sub(r"\n+", "\n", code)

In [ ]:
src_to_tgt = {}
slow_code_to_prompt = {}

for record in training_data:
    src_code = record['src_code']
    tgt_code = record['tgt_code']
    speedup = record['speedup']

    if src_code not in src_to_tgt or speedup > src_to_tgt[src_code]['speedup']:
        src_to_tgt[src_code] = record
        slow_code_to_prompt[src_code] = make_prompt(canonicalize_newlines(src_code), canonicalize_newlines(tgt_code))



In [ ]:
top_matches_pairs = []  # pair of (slow, fast)
test_train_pair = []
for i, top_match_idx in enumerate(top_matches):
    slow_code = training_data_src[top_match_idx]
    test_train_pair.append((test_set[i], src_to_tgt[slow_code]))




In [ ]:
inter_example_sep = "\n\n\n\n\n"

def _make_query(prompt, slow_code):
  prompt = f"""{prompt}




# slower version:

{slow_code}

# optimized version of the same code:"""
  return prompt





for i, test_set_ele in enumerate(test_set):
  # get the closest training examples
  top_matches_i = top_matches[i][0]
  closest_slow_codes = [training_data_src[idx] for idx in top_matches_i]
  closest_prompts = [slow_code_to_prompt[src_code] for src_code in closest_slow_codes]
  # test_set_ele["input"] = _make_query(closest_prompts[0], test_set_ele['src_code'])
  # test_set_ele["input"] = _make_query(inter_example_sep.join([closest_prompts[1], closest_prompts[0]]), test_set_ele['src_code'])
  test_set_ele["input"] = _make_query(inter_example_sep.join(closest_prompts[::-1]), test_set_ele['src_code'])


In [ ]:
with open("knnprompt_k4.jsonl", "w") as f:
    for test_set_ele in test_set:
        f.write(json.dumps(test_set_ele) + "\n")
